In [2]:
import os
import sys

#import matplotlib.pyplot as plt
#import numpy as np
import pandas as pd
import plotly.io as pio
#import seaborn as sns
#from feature_engine.selection import SmartCorrelatedSelection

import polars as pl

# Path needs to be added manually to read from another folder
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

In [3]:
data = pl.read_parquet('../data/supervised_clean_data.parquet')

In [4]:
data = data.drop(['What matters most to you in choosing a course','Search',
'Magazine','Newspaper Article','X Education Forums','Newspaper',
'Digital Advertisement','Through Recommendations','Receive More Updates About Our Courses',
'Update me on Supply Chain Content',
'Get updates on DM Content','I agree to pay the amount through cheque',
'A free copy of Mastering The Interview','Country'])

In [5]:
categorical_cols = [col for col in data.columns if data[col].dtype in [pl.Utf8, pl.Categorical]]

In [6]:
numerical_cols = [col for col in data.columns if data[col].dtype in [pl.Int32, pl.Int64, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]]

In [7]:

del numerical_cols[0]
del numerical_cols[0]
numerical_cols

['TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit']

In [8]:
numerical_cols

['TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit']

In [9]:
categorical_cols

['Prospect ID',
 'Lead Origin',
 'Lead Source',
 'Do Not Email',
 'Do Not Call',
 'Last Activity',
 'Specialization',
 'How did you hear about X Education',
 'What is your current occupation',
 'Tags',
 'Lead Quality',
 'Lead Profile',
 'City',
 'Last Notable Activity']

In [10]:
del categorical_cols[0]

In [10]:
data.select([
    pl.col(column).n_unique().alias(f"{column}_unique_count")
    for column in categorical_cols
])

Prospect ID_unique_count,Lead Origin_unique_count,Lead Source_unique_count,Do Not Email_unique_count,Do Not Call_unique_count,Last Activity_unique_count,Specialization_unique_count,How did you hear about X Education_unique_count,What is your current occupation_unique_count,Tags_unique_count,Lead Quality_unique_count,Lead Profile_unique_count,City_unique_count,Last Notable Activity_unique_count
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
9240,5,21,2,2,17,19,10,6,27,6,6,6,16


In [11]:
categorical_encoding_df = data.select(categorical_cols).to_dummies(drop_first=True)

In [29]:
categorical_encoding_df

Lead Origin_Landing Page Submission,Lead Origin_Lead Add Form,Lead Origin_Lead Import,Lead Origin_Quick Add Form,Lead Source_Click2call,Lead Source_Direct Traffic,Lead Source_Facebook,Lead Source_Google,Lead Source_Live Chat,Lead Source_NC_EDM,Lead Source_Organic Search,Lead Source_Pay per Click Ads,Lead Source_Press_Release,Lead Source_Reference,Lead Source_Referral Sites,Lead Source_Social Media,Lead Source_WeLearn,Lead Source_Welingak Website,Lead Source_bing,Lead Source_blog,Lead Source_google,Lead Source_testone,Lead Source_welearnblog_Home,Lead Source_youtubechannel,Do Not Email_Yes,Do Not Call_Yes,Last Activity_Approached upfront,Last Activity_Converted to Lead,Last Activity_Email Bounced,Last Activity_Email Link Clicked,Last Activity_Email Marked Spam,Last Activity_Email Opened,Last Activity_Email Received,Last Activity_Form Submitted on Website,Last Activity_Had a Phone Conversation,Last Activity_Olark Chat Conversation,Last Activity_Resubscribed to emails,…,Tags_Will revert after reading the email,Tags_in touch with EINS,Tags_invalid number,Tags_number not provided,Tags_opp hangup,Tags_switched off,Tags_wrong number given,Lead Quality_High in Relevance,Lead Quality_Might be,Lead Quality_None,Lead Quality_Not Sure,Lead Quality_Worst,Lead Profile_Dual Specialization Student,Lead Profile_Lateral Student,Lead Profile_Other Leads,Lead Profile_Potential Lead,Lead Profile_Student of SomeSchool,City_Other Cities,City_Other Cities of Maharashtra,City_Other Metro Cities,City_Thane & Outskirts,City_Tier II Cities,Last Notable Activity_Approached upfront,Last Notable Activity_Email Bounced,Last Notable Activity_Email Link Clicked,Last Notable Activity_Email Marked Spam,Last Notable Activity_Email Opened,Last Notable Activity_Email Received,Last Notable Activity_Form Submitted on Website,Last Notable Activity_Had a Phone Conversation,Last Notable Activity_Olark Chat Conversation,Last Notable Activity_Page Visited on Website,Last Notable Activity_Resubscribed to emails,Last Notable Activity_SMS Sent,Last Notable Activity_Unreachable,Last Notable Activity_Unsubscribed,Last Notable Activity_View in browser link Clicked
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,…,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,…,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,…,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [12]:
numerical_encoding_df = data.select(numerical_cols).select([
    (pl.col(col) - pl.col(col).min()) / (pl.col(col).max() - pl.col(col).min())
    for col in numerical_cols
])

In [31]:
numerical_encoding_df

TotalVisits,Total Time Spent on Website,Page Views Per Visit
f64,f64,f64
0.0,0.0,0.0
0.01992,0.296655,0.045455
0.007968,0.674296,0.036364
0.003984,0.134243,0.018182
0.007968,0.628521,0.018182
…,…,…
0.031873,0.81206,0.048545
0.007968,0.104754,0.036364
0.007968,0.087588,0.036364


In [13]:
#join all together
data = pl.concat([categorical_encoding_df,numerical_encoding_df, data.select('Converted')], how='horizontal')

In [13]:
data

Lead Origin_Landing Page Submission,Lead Origin_Lead Add Form,Lead Origin_Lead Import,Lead Origin_Quick Add Form,Lead Source_Click2call,Lead Source_Direct Traffic,Lead Source_Facebook,Lead Source_Google,Lead Source_Live Chat,Lead Source_NC_EDM,Lead Source_Organic Search,Lead Source_Pay per Click Ads,Lead Source_Press_Release,Lead Source_Reference,Lead Source_Referral Sites,Lead Source_Social Media,Lead Source_WeLearn,Lead Source_Welingak Website,Lead Source_bing,Lead Source_blog,Lead Source_google,Lead Source_testone,Lead Source_welearnblog_Home,Lead Source_youtubechannel,Do Not Email_Yes,Do Not Call_Yes,Last Activity_Approached upfront,Last Activity_Converted to Lead,Last Activity_Email Bounced,Last Activity_Email Link Clicked,Last Activity_Email Marked Spam,Last Activity_Email Opened,Last Activity_Email Received,Last Activity_Form Submitted on Website,Last Activity_Had a Phone Conversation,Last Activity_Olark Chat Conversation,Last Activity_Resubscribed to emails,…,Tags_opp hangup,Tags_switched off,Tags_wrong number given,Lead Quality_High in Relevance,Lead Quality_Might be,Lead Quality_None,Lead Quality_Not Sure,Lead Quality_Worst,Lead Profile_Dual Specialization Student,Lead Profile_Lateral Student,Lead Profile_Other Leads,Lead Profile_Potential Lead,Lead Profile_Student of SomeSchool,City_Other Cities,City_Other Cities of Maharashtra,City_Other Metro Cities,City_Thane & Outskirts,City_Tier II Cities,Last Notable Activity_Approached upfront,Last Notable Activity_Email Bounced,Last Notable Activity_Email Link Clicked,Last Notable Activity_Email Marked Spam,Last Notable Activity_Email Opened,Last Notable Activity_Email Received,Last Notable Activity_Form Submitted on Website,Last Notable Activity_Had a Phone Conversation,Last Notable Activity_Olark Chat Conversation,Last Notable Activity_Page Visited on Website,Last Notable Activity_Resubscribed to emails,Last Notable Activity_SMS Sent,Last Notable Activity_Unreachable,Last Notable Activity_Unsubscribed,Last Notable Activity_View in browser link Clicked,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Converted
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,f64,f64,f64,i64
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,…,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.01992,0.296655,0.045455,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,…,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.007968,0.674296,0.036364,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.003984,0.134243,0.018182,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.007968,0.628521,0.018182,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,…,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.031873,0.81206,0.048545,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.007968,0.104754,0.036364,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.007968,0.087588,0.036364,0


In [ ]:
#importance 

In [29]:
data

Lead Origin_Landing Page Submission,Lead Origin_Lead Add Form,Lead Origin_Lead Import,Lead Origin_Quick Add Form,Lead Source_Click2call,Lead Source_Direct Traffic,Lead Source_Facebook,Lead Source_Google,Lead Source_Live Chat,Lead Source_NC_EDM,Lead Source_Organic Search,Lead Source_Pay per Click Ads,Lead Source_Press_Release,Lead Source_Reference,Lead Source_Referral Sites,Lead Source_Social Media,Lead Source_WeLearn,Lead Source_Welingak Website,Lead Source_bing,Lead Source_blog,Lead Source_google,Lead Source_testone,Lead Source_welearnblog_Home,Lead Source_youtubechannel,Do Not Email_Yes,Do Not Call_Yes,Last Activity_Approached upfront,Last Activity_Converted to Lead,Last Activity_Email Bounced,Last Activity_Email Link Clicked,Last Activity_Email Marked Spam,Last Activity_Email Opened,Last Activity_Email Received,Last Activity_Form Submitted on Website,Last Activity_Had a Phone Conversation,Last Activity_Olark Chat Conversation,Last Activity_Resubscribed to emails,…,Tags_opp hangup,Tags_switched off,Tags_wrong number given,Lead Quality_High in Relevance,Lead Quality_Might be,Lead Quality_None,Lead Quality_Not Sure,Lead Quality_Worst,Lead Profile_Dual Specialization Student,Lead Profile_Lateral Student,Lead Profile_Other Leads,Lead Profile_Potential Lead,Lead Profile_Student of SomeSchool,City_Other Cities,City_Other Cities of Maharashtra,City_Other Metro Cities,City_Thane & Outskirts,City_Tier II Cities,Last Notable Activity_Approached upfront,Last Notable Activity_Email Bounced,Last Notable Activity_Email Link Clicked,Last Notable Activity_Email Marked Spam,Last Notable Activity_Email Opened,Last Notable Activity_Email Received,Last Notable Activity_Form Submitted on Website,Last Notable Activity_Had a Phone Conversation,Last Notable Activity_Olark Chat Conversation,Last Notable Activity_Page Visited on Website,Last Notable Activity_Resubscribed to emails,Last Notable Activity_SMS Sent,Last Notable Activity_Unreachable,Last Notable Activity_Unsubscribed,Last Notable Activity_View in browser link Clicked,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Converted
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,f64,f64,f64,i64
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,…,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.01992,0.296655,0.045455,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,…,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.007968,0.674296,0.036364,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.003984,0.134243,0.018182,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.007968,0.628521,0.018182,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,…,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.031873,0.81206,0.048545,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.007968,0.104754,0.036364,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.007968,0.087588,0.036364,0


In [14]:
scores = []
for f in remaining_engineered_features:
    print("Feature Analysis:", f)
    score = feature_predictive_power(graph_features, f, "is_anomaly")
    scores.append(score)

NameError: name 'remaining_engineered_features' is not defined

In [ ]:
pd.Series(scores, index=remaining_engineered_features).sort_values(ascending=False)

In [15]:
from sklearn.feature_selection import mutual_info_classif

In [16]:
data.select('Converted')

Converted
i64
0
0
1
0
1
…
1
0
0


In [17]:
importances = mutual_info_classif(data.drop('Converted'), data.select('Converted').to_numpy().flatten())

In [18]:
mutual_score = pd.DataFrame([ data.columns, importances], index = ["Features", "Importance"]).T.sort_values(by = "Importance", ascending = False).reset_index().drop("index", axis = 1)


In [19]:
low_score_col = mutual_score[mutual_score["Importance"] == 0].Features.values


In [24]:
low_score_col

array(['Lead Source_WeLearn', 'Lead Source_Referral Sites',
       'What is your current occupation_Housewife',
       'Lead Origin_Lead Import', 'City_Thane & Outskirts',
       'City_Tier II Cities', 'Last Notable Activity_Approached upfront',
       'Last Notable Activity_Email Bounced',
       'Last Notable Activity_Email Marked Spam',
       'How did you hear about X Education_Online Search',
       'Lead Source_Click2call', 'Last Notable Activity_Email Opened',
       'Lead Profile_Other Leads',
       'What is your current occupation_Businessman',
       'Specialization_Retail Management',
       'How did you hear about X Education_SMS',
       'Specialization_Travel and Tourism',
       'Last Notable Activity_View in browser link Clicked',
       'Tags_Diploma holder (Not Eligible)', 'Lead Source_youtubechannel',
       'Lead Source_testone', 'Tags_Still Thinking',
       'Tags_In confusion whether part time or DLP',
       'Specialization_Human Resource Management',
       'Sp

In [20]:
high_score_col = mutual_score[mutual_score["Importance"] > 0].Features.values


In [25]:
high_score_col

array(['Tags_Will revert after reading the email',
       'Total Time Spent on Website', 'Lead Quality_None',
       'Lead Profile_Potential Lead', 'Last Notable Activity_SMS Sent',
       'Lead Quality_Might be', 'Lead Quality_High in Relevance',
       'Last Activity_SMS Sent',
       'What is your current occupation_Working Professional',
       'Tags_Ringing', 'Lead Origin_Lead Add Form',
       'Lead Source_Reference', 'Tags_Closed by Horizzon',
       'Last Activity_Olark Chat Conversation', 'Tags_Already a student',
       'Lead Quality_Worst', 'Tags_None', 'Page Views Per Visit',
       'Tags_opp hangup', 'Tags_Lost to EINS',
       'Lead Profile_Student of SomeSchool', 'Tags_switched off',
       'Lead Source_bing', 'Do Not Email_Yes',
       'Lead Source_Welingak Website',
       'Lead Profile_Dual Specialization Student',
       'Lead Source_Direct Traffic',
       'Specialization_Banking, Investment And Insurance',
       'Last Activity_Converted to Lead', 'TotalVisits',
  

In [26]:
feature_data = data.select(high_score_col)


In [27]:
data = pl.concat([feature_data, data.select('Converted')], how='horizontal')


In [ ]:
def cleaning_pipeline(data: pl.DataFrame, output_path: str):
    """Data cleaning and processing pipeline

    Args:
        data (pl.DataFrame): input dataset that needs to be pre-processed
        output_path (str): location of where to save the parquet file
    """

    
    # Drop unnecessary columns
    columns_to_drop = [
        'What matters most to you in choosing a course','Search','Magazine','Newspaper Article','X Education Forums','Newspaper',
'Digital Advertisement','Through Recommendations','Receive More Updates About Our Courses',
'Update me on Supply Chain Content',
'Get updates on DM Content','I agree to pay the amount through cheque',
'A free copy of Mastering The Interview','Country'
    ]
    data = data.drop(columns_to_drop)

    categorical_cols = ['Lead Origin','Lead Source','Do Not Email','Do Not Call','Last Activity','Specialization',
 'How did you hear about X Education','What is your current occupation','Tags','Lead Quality','Lead Profile',
 'City','Last Notable Activity']

    numerical_cols = ['TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit']

    categorical_encoding_df = data.select(categorical_cols).to_dummies(drop_first=True)

    numerical_encoding_df = data.select(numerical_cols).select([
    (pl.col(col) - pl.col(col).min()) / (pl.col(col).max() - pl.col(col).min())
    for col in numerical_cols
])

    data = pl.concat([categorical_encoding_df,numerical_encoding_df, data.select('Converted')], how='horizontal')

    

    low_importance = ['Lead Source_WeLearn', 'Lead Source_Referral Sites',
       'What is your current occupation_Housewife',
       'Lead Origin_Lead Import', 'City_Thane & Outskirts',
       'City_Tier II Cities', 'Last Notable Activity_Approached upfront',
       'Last Notable Activity_Email Bounced',
       'Last Notable Activity_Email Marked Spam',
       'How did you hear about X Education_Online Search',
       'Lead Source_Click2call', 'Last Notable Activity_Email Opened',
       'Lead Profile_Other Leads',
       'What is your current occupation_Businessman',
       'Specialization_Retail Management',
       'How did you hear about X Education_SMS',
       'Specialization_Travel and Tourism',
       'Last Notable Activity_View in browser link Clicked',
       'Tags_Diploma holder (Not Eligible)', 'Lead Source_youtubechannel',
       'Lead Source_testone', 'Tags_Still Thinking',
       'Tags_In confusion whether part time or DLP',
       'Specialization_Human Resource Management',
       'Specialization_Hospitality Management', 'Tags_Lateral student',
       'Specialization_Finance Management', 'Specialization_E-Business',
       'Tags_Not doing further education',
       'Last Activity_Visited Booth in Tradeshow',
       'Last Activity_View in browser link Clicked',
       'Tags_Shall take in the next coming month',
       'Last Activity_Unsubscribed', 'Tags_Graduation in progress',
       'Specialization_IT Projects Management', 'Tags_invalid number',
       'Tags_number not provided', 'Tags_Busy',
       'Last Activity_Form Submitted on Website',
       'Tags_wrong number given', 'Last Activity_Email Received',
       'Specialization_Marketing Management',
       'Last Activity_Approached upfront', 'Do Not Call_Yes',
       'How did you hear about X Education_Advertisements']
    
    

    

    numerical_encoding_df = data.select(numerical_cols).select([
    (pl.col(col) - pl.col(col).min()) / (pl.col(col).max() - pl.col(col).min())
    for col in numerical_cols
])

    data.write_parquet(output_path)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
train_data, test_data = train_test_split(data, test_size = 0.25)


In [30]:
output_train = "../data/supervised_train_data.parquet"
output_test = "../data/supervised_test_data.parquet"

train_data.write_parquet(output_train)
test_data.write_parquet(output_test)
#cleaning_pipeline(data, output)

In [32]:
data.select('Converted').to_numpy().flatten()

array([0, 0, 1, ..., 0, 1, 1], dtype=int64)